In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.ndimage
import cv2
import scipy.misc

In [ ]:
def ECC(im1,im2,iterations=100):
    # Convert images to grayscale
    im1_gray = cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY)
    im2_gray = cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)

    # Find size of image1
    sz = im1.shape

    # Define the motion model
    warp_mode = cv2.MOTION_EUCLIDEAN

    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Specify the number of iterations.
    number_of_iterations = iterations;

    # Specify the threshold of the increment
    # in the correlation coefficient between two iterations
    termination_eps = 1e-10;

    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    # Run the ECC algorithm. The results are stored in warp_matrix.
    (cc, warp_matrix) = cv2.findTransformECC (im1_gray,im2_gray,warp_matrix, warp_mode, criteria)

    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        # Use warpPerspective for Homography 
        im2_aligned = cv2.warpPerspective (im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else :
        # Use warpAffine for Translation, Euclidean and Affine
        im2_aligned = cv2.warpAffine(im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

    return im2_aligned

In [ ]:
root = "../Images/"
where_to_save = "../ImagesRec/"
start_idx=0
end_idx=1000
name_mean = 'mean_image_{}-{}.jpg'.format(start_idx,end_idx)
paths = os.listdir(root)[:end_idx]
compute = True
mean_image = scipy.ndimage.imread('mean_image.jpg')
base_image = scipy.ndimage.imread(root+paths[0])

In [ ]:
def compute_mean(paths,processing=True):
    sum_array = None
    w = 0
    h = 0
    i=0
    for file in paths:
        i = i+1
        ndarray = scipy.ndimage.imread(root+file)
        if(processing is True):
            norm_a = ECC(base_image,ndarray,iterations=300)
            scipy.misc.imsave(where_to_save+file, norm_a)
        else:
            norm_a = ndarray
        norm_a = norm_a*1.0/len(paths)
        if(sum_array is None):
            sum_array = norm_a
        else:
            sum_array = np.add(sum_array,norm_a)
        if(i%10==0):
            print(i)
    return sum_array

In [ ]:
from threading import Thread
import time
from multiprocessing import Queue

#Multiprocessing mean
def par_compute_mean(i,paths, queue):
    result = compute_mean(paths)
    queue.put({i: result})

def compute_mean_image(paths,number_of_threads = 20):
    q = Queue()
    threads = []
    
    step = len(paths)/number_of_threads
    arguments = []
    
    for i in range(number_of_threads):
        start = int(i*step)
        end = int((i+1)*step)
        arg = paths[start:end]
        arguments.append((i,arg))
    
    for i,arg in arguments:
        t = Thread(target=par_compute_mean, args=(i,arg, q))
        t.start()
        threads.append(t)
        
    for t in threads:
        t.join()
    
    total_means = [q.get() for _ in range(len(arguments))]

    means = []

    for d in total_means:
        k = list(d.keys())[0]
        val = d[k]
        means.append(val)
        
    return np.mean(means,axis=0)

In [ ]:
import scipy.misc
mean_image_computed = compute_mean_image(paths[:5000],number_of_threads=6)
scipy.misc.imsave(name_mean, mean_image_computed)